In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import TimesNet
from load_data import load_data
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 

In [2]:
train_imu,train_audio,train_spec,val_imu,val_audio,val_spec,val_imu_normal,val_audio_normal,val_spec_normal = load_data()

train_combine = np.concatenate([np.mean(train_audio,axis=-1),train_imu],axis=1)

test_audio_all = np.concatenate([val_audio_normal,val_audio],axis=0)
test_imu_all   = np.concatenate([val_imu_normal,val_imu],axis=0)
test_combine = np.concatenate([np.mean(test_audio_all,axis=-1),test_imu_all],axis=1)

labels_all = np.concatenate([np.zeros(val_audio_normal.shape[0]),np.ones(val_audio.shape[0])])

In [14]:
clf_imu = TimesNet(seq_len=10,epochs=50)
clf_imu.fit(train_imu)
clf_audio = TimesNet(seq_len=10,epochs=50)
clf_audio.fit(np.mean(train_audio,axis=-1))
clf_all = TimesNet(seq_len=10,epochs=50)
clf_all.fit(train_combine)

RuntimeError: shape '[32, 0, 0, 64]' is invalid for input of size 18432

In [6]:
scores_audio      = clf_audio.decision_function(np.mean(test_audio_all,axis=-1))
scores_imu        = clf_imu.decision_function(test_imu_all)
scores_all      = clf_all.decision_function(test_combine)

In [7]:
#auc, mean precision, F1, percision, recall
eval_metrics_audio = ts_metrics(labels_all, scores_audio)
eval_metrics_imu = ts_metrics(labels_all, scores_imu)
result_audio = eval_metrics_audio
result_imu = eval_metrics_imu
eval_metrics_all = ts_metrics(labels_all, scores_all)
result_all = eval_metrics_all

print(result_audio)
print(result_imu)
print(result_all)

(0.9754044930238983, 0.9574250757872725, 0.9173776837019793, 0.9241011984021305, 0.910761154855643)
(0.8701057207487222, 0.8659649569712555, 0.794352918584039, 0.9145299145299145, 0.7020997375328084)
(0.9451905477275867, 0.9264831308518484, 0.8517974047398402, 0.8668478260869565, 0.8372703412073491)
